In [1]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install optuna

!pip install optuna

In [3]:
# 2. Install Ultralytics YOLO if not installed
!pip install ultralytics

In [4]:
# 3. Import required modules
import zipfile
import os
from ultralytics import YOLO
import optuna

In [5]:
# 4. Define paths
train_zip = '/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/train.zip'
val_zip = '/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/val.zip'
unzip_dir = '/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset'

In [6]:
# 5. Unzip train and val if not already unzipped
def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

if not os.path.exists(os.path.join(unzip_dir, 'train')):
    unzip_file(train_zip, unzip_dir)

if not os.path.exists(os.path.join(unzip_dir, 'val')):
    unzip_file(val_zip, unzip_dir)

In [7]:
# 6. Path to your config.yaml
data_yaml = '/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/config.yaml'

In [8]:
def objective(trial):
    from ultralytics import YOLO

    # Suggest valid hyperparameters
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-2, log=True)  # correct: initial learning rate
    batch = trial.suggest_categorical("batch", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    imgsz = trial.suggest_categorical("imgsz", [320, 416, 512, 640])
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    mosaic = trial.suggest_float("mosaic", 0.0, 1.0)
    mixup = trial.suggest_float("mixup", 0.0, 1.0)

    # Load model
    model = YOLO("yolov8n-seg.pt")

    # Train model
    results = model.train(
        data=data_yaml,
        epochs=100,
        imgsz=imgsz,
        batch=batch,
        lr0=lr0,
        weight_decay=weight_decay,
        dropout=dropout,
        mosaic=mosaic,
        mixup=mixup,
        optimizer="Adam",
        project="/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/runs",
        name=f"trial_{trial.number}",
        exist_ok=True,
        verbose=False
    )

    # Validate model and return the metric to optimize
    metrics = model.val()
    return metrics.seg.map50  # Return mAP@0.5 for segmentation


In [ ]:
# Run the Optuna study
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best hyperparameters found:")
print(study.best_trial.params)



[I 2025-05-31 21:48:17,893] A new study created in memory with name: no-name-ced2f35f-4dc4-49b0-ae97-3698e7aadddd


Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.1791728939555941, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00013332666408553015, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.6963494128346619, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=0.30760535850607384, multi_scale=False, name=trial_0, nbs=64, nms=False

train: Scanning /content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/train/labels.cache... 1100 images, 1100 backgrounds, 0 corrupt: 100%|██████████| 2200/2200 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 3.4±5.6 ms, read: 1.8±0.9 MB/s, size: 3.0 KB)


val: Scanning /content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/val/labels.cache... 300 images, 300 backgrounds, 0 corrupt: 100%|██████████| 600/600 [00:00<?, ?it/s]


...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      7/100      1.29G     0.6377      1.335     0.7591     0.9781         29        320: 100%|██████████| 69/69 [00:56<00:00,  1.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07<00:00,  1.42it/s]

                   all        600        300      0.996       0.99      0.994      0.951      0.996       0.99      0.994      0.947



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      8/100      1.29G     0.6522       1.41     0.7536     0.9963         27        320:  62%|██████▏   | 43/69 [00:36<00:21,  1.23it/s]